# Sequence Assembly

In diesem Praktikum geht es um das Assembly von DNA-Sequenzen – optimalerweise aus Ihrem Praktikum. Da wir dort aber verschiedene Versuche fahren, arbeiten wir hier mit simulierten Reads.

## Vorbereitungen

Ich gehe davon aus, dass Sie in Ihrem Home-Verzeichnis einen Ordner namens _Assembly_ haben. Das ist unser Startpunkt.

In [ ]:
%cd ~/Assembly

### Install Unicycler

Unicycler ist ein Programm, das viele Abhängigkeiten aufweist. Daher müssen wir zunächst eine Reihe von Programmen installieren, bevor wir loslegen können:

- spades -> für das Assembly kurzer Reads
- racon -> für das Assembly langer Reads
- makeblastdb -> für das Handling zikularer Sequenzen     
- tblastn -> für das Handling zikularer Sequenzen     
- bowtie2-build -> für die Qualitätskontrolle
- bowtie2 -> für die Qualitätskontrolle
- samtools -> für die Qualitätskontrolle
- pilon -> für die Qualitätskontrolle
- bcftools -> für die Qualitätskontrolle 
- java -> als Programmiersprache   
- python2 -> als Programmiersprache

Aber legen wir erstmal mit dem Unicycler los:

In [ ]:
!pkexec sudo apt-get -y install git | tail -3

In [ ]:
!git clone https://github.com/rrwick/Unicycler.git

In [ ]:
cd Unicycler

In [ ]:
!pkexec sudo apt-get install libz-dev | tail -3

In [ ]:
!make

In [ ]:
!./unicycler-runner.py

In [ ]:
%cd ..

### Install Racon

In [ ]:
!git clone --recursive https://github.com/isovic/racon.git # racon

In [ ]:
%cd racon

In [ ]:
%mkdir build

In [ ]:
%cd build

In [ ]:
!pkexec apt-get -y install cmake | tail -3

In [ ]:
!cmake -DCMAKE_BUILD_TYPE=Release ..

In [ ]:
!make

In [ ]:
!pkexec make install

In [ ]:
%cd ~/Assembly

### Install Spades

**Wichtig**: Versionen über 3.13 sind zu neu und funktionieren nicht.

In [ ]:
!wget http://cab.spbu.ru/files/release3.13.0/SPAdes-3.13.0-Linux.tar.gz

In [ ]:
!tar -xf SPAdes-3.13.0-Linux.tar.gz

### Install BLAST+

In [ ]:
!pkexec apt-get  install -y ncbi-blast+ | tail -3

### Install Samtools

In [ ]:
!pkexec apt-get -y install samtools

### Install Bowtie2

In [ ]:
!pkexec apt-get -y install bowtie2 | tail -3

### Install Pilon

In [ ]:
!wget https://github.com/broadinstitute/pilon/releases/download/v1.23/pilon-1.23.jar

## Install Sequencer

Sequencer ist ein Tool von Bernhard Haubold, das einen Sequenzierautomaten simuliert: Man gibt ein Genom rein und bekommt Reads heraus. Dabei kann man die durchschnittliche gewünschte Coverage und Readlänge vorgeben. Um die Mittelwerte wird eine Gauss'sche Verteilung erzeugt.

In [ ]:
%cd /home/rw/Assembly

In [ ]:
!wget 'http://www.staff.hs-mittweida.de/~wuenschi/data/media/compbiolbook/sequencer'

In [ ]:
!chmod u+x sequencer

In [ ]:
!pkexec apt-get install lib32z1 # for sequencer

In [ ]:
!./sequencer

## Run Assembly

In [ ]:
!time Unicycler/unicycler-runner.py -s 30-150.fastq -o output-30-150 --spades_path /home/rw/Assembly/SPAdes-3.13.0-Linux/bin/spades.py --pilon_path /home/rw/Assembly/pilon-1.23.jar

## Run Assembly / Datenanalyse

Zunächst erzeugen wir mit dem virtuellen Sequenzer Reads des SARS-CoV2-Genoms. Die Option -c gibt die Coverage und -l die Readlänge an. Wir speichern sie in den Pythonvariablen _cov_ und _len_. Mit {Variabelname} können wir sie in der Shell aufrufen. Das ist total cool :-)

Mit _env_ exportieren wir die Variablen. Sie stehen sie auch Subshells zur Verfügung.

#### AUFGABE
Probieren Sie verschiedene Coverage/Readlängen-Kombinationen und stellen Sie graphisch dar (R?), wie sich das auf die Anzahl der Contigs auswirkt.

In [ ]:
cov = 9
len = 188

In [ ]:
%env cov={cov}
%env len={len}

In [ ]:
!echo "$cov+$len"

In [ ]:
!echo {cov}"+"{len}

In [ ]:
!./sequencer -i cov2.fasta -c {cov} -l {len} -o cov2-{cov}-{len}.out

In [ ]:
!grep -c ">" cov2-{cov}-{len}.out

AWK braucht die Shellvariabeln mit dem Dollarzeichen ...

In [ ]:
!awk -f fast2tab.awk "cov2-$cov-$len.out" | awk 'length($2)>0{print "@"$1"\n"$2"\n+";for(i=1; i<=length($2); i++){ORS="";print"X";ORS="\n"}print "\n"}' | sed '/^$/d' > "cov2-$cov-$len.fastq"

In [ ]:
!time Unicycler/unicycler-runner.py -s cov2-{cov}-{len}.fastq -o output-cov2-{cov}-{len} --spades_path ~/Assembly/SPAdes-3.13.0-Linux/bin/spades.py --pilon_path ~/Assembly/pilon-1.23.jar

In [ ]:
!grep -c ">" output-cov2-{cov}-{len}/assembly.fasta